# Importing Libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 

import pandas as pd
import time
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as reg
import pickle
import pandas as pd
import itertools

from explicit import waiter, XPATH


### Disable images

In [ ]:
option = webdriver.ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}

driver = webdriver.Chrome(options=option,executable_path=r'path')

### Simple Chrome 

In [ ]:
driver = webdriver.Chrome(executable_path=r'path')

In [ ]:
def login(username, password):
    driver.get('https://www.instagram.com/accounts/login/?source=auth_switcher')
    # //*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[2]/div/label/input
    time.sleep(2)
    driver.find_element_by_name('username').send_keys(username)
    driver.find_element_by_name('password').send_keys(password)
    driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div[4]/button/div').click()

    driver.maximize_window()
    
username = 'username'
password = 'password'
login(username,password)



###  Open instagram account and get 
##### Number of Posts
##### Followers
##### Following

In [ ]:
account ='global.influencers'
driver.get("https://www.instagram.com/{0}".format(account))

page_soup = soup(driver.page_source,'lxml')
posts = page_soup.findAll('span',{'class','g47SY'})
post = posts[0].text
followings = posts[2].text
followers = posts[1].get('title')


### Scrape All Followers 

In [ ]:
def scrape_followers(driver, account):
    # Load account page
    driver.get("https://www.instagram.com/{0}/".format(account))

    followersLink = driver.find_element_by_css_selector('ul li a')
    followersLink.click()
    time.sleep(2)

    waiter.find_element(driver, "//div[@role='dialog']", by=XPATH)

    follower_css = "ul div li:nth-child({}) a.notranslate"  # Taking advange of CSS's nth-child functionality
    for group in itertools.count(start=1, step=12):
        for follower_index in range(group, group + 12):
            yield waiter.find_element(driver, follower_css.format(follower_index)).text

        last_follower = waiter.find_element(driver, follower_css.format(follower_index))
        driver.execute_script("arguments[0].scrollIntoView();", last_follower)
        
Followers = []
for count, follower in enumerate(scrape_followers(driver, account=account), 1):
    Data = {}
    Data ['Followers'] = "https://www.instagram.com/"+follower
    Followers.append(Data)
    if count >= int(followers):
        break

### Scrape All Followings

In [ ]:
def scrape_followings(driver, account):
    # Load account page
    driver.get("https://www.instagram.com/{0}/".format(account))

    followersLink = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/header/section/ul/li[3]/a')
    followersLink.click()
    time.sleep(2)

    waiter.find_element(driver, "//div[@role='dialog']", by=XPATH)

    follower_css = "ul div li:nth-child({}) a.notranslate"  # Taking advange of CSS's nth-child functionality
    for group in itertools.count(start=1, step=12):
        for follower_index in range(group, group + 12):
            yield waiter.find_element(driver, follower_css.format(follower_index)).text

        last_follower = waiter.find_element(driver, follower_css.format(follower_index))
        driver.execute_script("arguments[0].scrollIntoView();", last_follower)

Followings = []
for count, following in enumerate(scrape_followings(driver, account=account), 1):
    Data = {}
    Data['Followings'] = "https://www.instagram.com/"+following
    Followings.append(Data)
    if count >= int(followings):
        break

### Scrape All Posts 

In [ ]:
def scrape_Posts_URL(driver , account):
    
    page_soup = soup(driver.page_source,'lxml')
    Posts_links = []
    Tagged_in_Posts = page_soup.findAll('div',{'class','Nnq7C weEfm'})

    i = 0
    while(i <int(post)):
        print(len(Tagged_in_Posts),i)
        i+=len(Tagged_in_Posts)
        for posts in Tagged_in_Posts:
            post_links  = posts.findAll('a') 
            for link in post_links:
                Data ={}
                Data['Post Links'] = 'https://www.instagram.com/'+link.get('href')
                Posts_links.append(Data)            

        scroll = driver.find_element_by_tag_name('html')
        scroll.send_keys(Keys.END)
        time.sleep(2)

        Tagged_in_Posts = page_soup.findAll('div',{'class','Nnq7C weEfm'})
        page_soup = soup(driver.page_source,'lxml')

    return Posts_links
    
posts_url = scrape_Posts_URL(driver,account)

### Scrape All Tagged In

In [ ]:
def scrape_Tagged_in_Posts_URL(driver , account):
    driver.get("https://www.instagram.com/"+account+'/tagged')
    page_soup = soup(driver.page_source,'lxml')
    Posts_links = []
    Tagged_in_Posts = page_soup.findAll('div',{'class','Nnq7C weEfm'})
    time.sleep(2)
    flag = True
    while(flag):
        print(len(Tagged_in_Posts))
        if(flag==True):
            for posts in Tagged_in_Posts:
                post_links  = posts.findAll('a') 
                if(flag==True):
                    for link in post_links:
                        Data ={}
                        url = 'https://www.instagram.com/'+link.get('href')
                        Data['Post Links'] = url
                        Posts_links.append(Data)
                        if(url == 'https://www.instagram.com//p/BeKwrKDFLq9/'):
                            flag = False

        scroll = driver.find_element_by_tag_name('html')
        scroll.send_keys(Keys.END)
        time.sleep(2)

        Tagged_in_Posts = page_soup.findAll('div',{'class','Nnq7C weEfm'})
        page_soup = soup(driver.page_source,'lxml')

    return Posts_links

tagged_posts_url = scrape_Tagged_in_Posts_URL(driver,account)